[Reference](https://medium.com/@dhruvahuja2330/async-concurrent-why-90-of-python-developers-get-concurrency-wrong-0a79dfae362e)

In [1]:
import asyncio

async def greet():
    print("Hello")
    await asyncio.sleep(1)   # pause here
    print("World")

greet()

<coroutine object greet at 0x7be0bd574ac0>

In [3]:
import asyncio, time

async def fetch_data(url):
    print(f"Fetching {url}")
    await asyncio.sleep(2)
    return f"Data from {url}"

async def main():
    start = time.time()
    result1 = await fetch_data("https://api1.com")
    result2 = await fetch_data("https://api2.com")
    result3 = await fetch_data("https://api3.com")
    end = time.time()
    print(f"Total time: {end - start:.2f}s")

asyncio.run(main())

# Scheduling With create_task()

In [4]:
import asyncio, time

async def fetch_data(url):
    print(f"Fetching {url}")
    await asyncio.sleep(2)
    return f"Data from {url}"

async def main():
    start = time.time()
    task1 = asyncio.create_task(fetch_data("https://api1.com"))
    task2 = asyncio.create_task(fetch_data("https://api2.com"))
    task3 = asyncio.create_task(fetch_data("https://api3.com"))
    results = await asyncio.gather(task1, task2, task3)
    end = time.time()
    print(f"Total time: {end - start:.2f}s")
    print("Results:", results)

asyncio.run(main())

In [5]:
import asyncio, aiohttp
from asyncio import Semaphore

class APIClient:
    def __init__(self, max_concurrent=5):
        self.sem = Semaphore(max_concurrent)

    async def fetch(self, session, uid):
        async with self.sem:   # only N tasks allowed at once
            async with session.get(f"https://api.com/users/{uid}") as resp:
                return await resp.json()

    async def fetch_all(self, uids):
        async with aiohttp.ClientSession() as s:
            tasks = [asyncio.create_task(self.fetch(s, uid)) for uid in uids]
            return await asyncio.gather(*tasks)

client = APIClient(max_concurrent=10)
results = asyncio.run(client.fetch_all(range(50)))